## Library imports

In [2]:
import numpy as np
import pandas as pd

## Dataset Import

In [3]:
alist = pd.read_csv('../data/Peter32.csv')

## Useful functions definition

In [4]:
def sig(height, mu):
    return np.sqrt((mu^2) * height)

In [5]:
def pl_error(result_list):
    list_len = result_list.shape[0]
    true_positions = np.arange(1, list_len + 1)/list_len
    positions = result_list[:, 0]
    return np.abs(positions - true_positions).sum() / (0.5 * (list_len - 1))

In [6]:
def k(k0, S, phi):
    return (10 ** (-phi * S)) * k0

In [7]:
def xk(k0, S, phi, dt):
    return dt /(k(k0, S, phi) + 1)

In [8]:
def iso_list(anlist, phi):
    k0 = anlist[:,1]
    S = anlist[:,2]
    dt =  (1 - 2 * np.sqrt(0.001))
    return np.sort(dt /((10 ** (-phi * S)) * k0 + 1))

In [9]:
def iso_spot(anlist, phi):
    return pl_error(iso_list(anlist, phi))

In [10]:
def int_(ud, sigd, ua, siga):
    return (2 * siga ** 2 * sigd - 2 * siga * sigd ** 2 \
            + sigd ** 2 * ua + sigda ** 2 * ud)\
            / ( siga ** 2 + sigd ** 2)


In [11]:
def bl(ud, sigd, ua, siga):
    first = int_(ud, sigd, ua, siga)
    second = ua - 2 * siga
    return np.stack([first,second], axis=1).max(axis=1)

In [12]:
def bu(ud, sigd, ua, siga):
    first = int_(ud, sigd, ua, siga)
    second = ud + 2 * sigd
    return np.stack([first,second], axis=1).min(axis=1)

In [13]:
def cushlist(res_list):
    return np.concatenate([ np.array([
        0, 10 ** -10]).reshape(1, -1),
        res_list,
        np.array([1, 10 ** -10]).reshape(1, -1)])

In [44]:
def poi(res_list):
    np.stack(
        [
            bl(res_list[:-2, 0], res_list[:-2, 1], res_list[1:-1, 0], res_list[1:-1, 1]),
            res_list[1:-1, 0],
            bu(res_list[1:-1, 0], res_list[1:-1, 1], res_list[2:, 0], res_list[2:, 1])
        ],
        axis=1 ###???
       )

In [50]:
def yval(x_list, chrom):

    denominator = 4 * chrom[:, 1] ** 2
    temp_1 = np.abs(x_list[:, 0] - x_list[:1])
    temp_1[temp_1 > chrom[:1]] = 0
    temp_1 /= denominator
    
    temp_2 = 1/(2 * chrom[:, 1])
    
    temp_3 = np.abs(x_list[:, 1] - x_list[:, 1])
    temp_3[np.abs(x_list[:, 2] - x_list[:1]) > chrom[:1]] = 0
    temp_3 /= denominator
    
    return np.stack([temp_1, temp_2, temp_3], axis=1)


In [51]:
def narea(main_list, x_list, y_list):
    return 1 - ((y_list[:, 1] - y_list[:, 0])*(x_list[:, 1] - x_list[:, 0])
            - (y_list[:, 2] - y_list[:, 1])*(x_list[:, 2] - x_list[:, 1]))/(2 * main_list.shape[0])

In [52]:
def lfn(main_list, x_list, y_list):
    return plerror(main_list) + narea(main_list, x_list, y_list)

In [ ]:
def isoloss(an_list, phi, h):
    pass